In [10]:
## TODO execute in docker container -> no network connection
## Query time with time or rather explain?
##TODO shift ppi to syn_data (new mount for import dir)

In [1]:
import subprocess
import docker
import os
import neo4j
from neo4j import GraphDatabase
import pandas as pd
import time
import numpy as np
import time
from tqdm.notebook import tqdm
from torch_geometric.utils import sort_edge_index
import torch
from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader

/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# ppi = PPI(root = "data")
# data_loader = iter(DataLoader(ppi, batch_size=len(ppi)))
# ppi_data = next(data_loader)
# pd.DataFrame(ppi_data.x).to_csv("data/ppi_x.csv", index = False, sep = ",")
# pd.DataFrame(ppi_data.y).to_csv("data/ppi_y.csv", index = False, sep = ",")
# pd.DataFrame(ppi_data.edge_index.transpose(-1, 0)).to_csv("data/ppi_edge_index.csv", index = False, sep = ",")

In [12]:
def create_neo4j_container():
    # Create a Docker client
    client = docker.from_env()
    
    # Define the image name
    image_name = "neo4j:latest"
    
    # Pull the image if it is not already available
    client.images.pull(image_name)
    
    # Define the container name and environment variables
    container_name = "neo4j-apoc"
    neo4j_env = {
        "NEO4J_apoc_export_file_enabled": "true",
        "NEO4J_apoc_import_file_enabled": "true",
        "NEO4J_apoc_import_file_use__neo4j__config": "true",
        "NEO4J_PLUGINS": '["apoc"]',
        "NEO4J_AUTH": "neo4j/password",
    }
    
    # Define the volume mapping
    volume_mapping = {
        f"{os.getcwd()}/neo4j_data": {"bind": "/data", "mode": "rw"},
        f"{os.getcwd()}/syn_data": {"bind": "/import", "mode": "rw"}
    }
    ports = {"7474": 7475, "7687": 7688}
    # Run the container
    container = client.containers.run(
        image=image_name,
        name=container_name,
        ports=ports,
        environment=neo4j_env,
        volumes=volume_mapping,
        detach=True,
        tty=True,
        stdin_open=True,
        remove=True,  # Automatically remove the container when it stops
    )
    
    print(f"Container {container_name} started with ID: {container.id}")
    return container, ports

In [13]:
container, ports = create_neo4j_container()

Container neo4j-apoc started with ID: df12e7bcc73fb46e95a7e7180efae9654955d6549eae455a704169990cf17d40


In [5]:
#container.stop()

In [14]:
def connect_to_neo4j(uri, user, password):
    """
    Establishes a connection to the Neo4j database.

    Args:
        uri (str): The URI of the Neo4j database.
        user (str): The username for authentication.
        password (str): The password for authentication.

    Returns:
        GraphDatabase.driver: The Neo4j driver instance for the connection.
    """

    return GraphDatabase.driver(uri, auth=(user, password))

In [16]:
# uri = f"bolt://localhost:{ports['7687']}"
uri = f"bolt://localhost:{7688}"
username = "neo4j"
password = "password"

driver = connect_to_neo4j(uri, username, password)
driver.execute_query("MATCH (n) DETACH DELETE n")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7ff15bc583a0>, keys=[])

In [ ]:
# data_root_dir = "/home/dwalke/git/shweta/Master-Thesis-Code/data/syn_data"
# num_nodes = 10000
# num_edges = 5
# # edge_file_name = f"edge_index_{num_nodes}_nodes_{num_edges}_edges.csv"
# # X = pd.read_csv(f"{data_root_dir}/X_{num_nodes}_nodes_{num_edges}_edges.csv")
# # y = pd.read_csv(f"{data_root_dir}/y_{num_nodes}_nodes_{num_edges}_edges.csv")
# # edges = pd.read_csv(f"{data_root_dir}/{edge_file_name}")
# edge_file_name = "ppi_edge_index.csv"
# X = pd.read_csv(f"data/ppi_x.csv")
# y = pd.read_csv(f"data/ppi_y.csv")
# edges = pd.read_csv("data/" + edge_file_name)
# edge_index = edges.values.astype(np.int64).transpose()
# X_and_y = pd.DataFrame()
# X_and_y["label"] = y.apply(lambda row: [row[column] for column in y.columns], axis=1)
# X_and_y["X"] = X.apply(lambda row: [row[column] for column in X.columns], axis=1)
# node_file_name = "X_y_ppi_neo4j.csv"
# X_and_y.to_csv(f"data/{node_file_name}", sep = ",", index = False)

In [28]:
def recurse_edge_index_iterative(source_nodes, edge_index, max_depth):
    """
    Optimized function to compute the subgraph around the source nodes up to a given depth.
    Uses an iterative approach instead of recursion.
    """
    visited_nodes = set(source_nodes)
    current_frontier = np.array(source_nodes)
    
    subgraph_edges = []

    for _ in range(max_depth):
        # Find edges where the target node is in the current frontier
        target_mask = np.isin(edge_index[1], current_frontier)
        subgraph_edge_index = edge_index[:, target_mask]
        subgraph_edges.append(subgraph_edge_index)

        # Update the current frontier with the source nodes of these edges
        current_frontier = np.setdiff1d(subgraph_edge_index[0], list(visited_nodes))
        visited_nodes.update(current_frontier)
        
        if len(current_frontier) == 0:
            break

    # Combine edges from all hops
    return np.concatenate(subgraph_edges, axis=1) if subgraph_edges else np.empty((2, 0), dtype=edge_index.dtype)


def get_subgraph_from_in_mem_graph_optimized(X, y, i, edge_index, hops):
    """
    Optimized version of subgraph extraction.
    """
    subgraph_edge_index = recurse_edge_index_iterative([i], edge_index, hops)
    unique_node_ids, remapping = np.unique(subgraph_edge_index, return_inverse=True)
    
    # Extract features and labels
    features = X.iloc[unique_node_ids, :].values
    labels = y.iloc[unique_node_ids, :].values.squeeze()

    # Remap edge indices
    remapped_edge_index = remapping.reshape(2, -1)
    return remapped_edge_index, features, labels, unique_node_ids
    
def create(driver, node_file_name, edge_file_name):
    with driver.session() as session:
        start = time.time()
        node_query = """
                LOAD CSV WITH HEADERS FROM $file AS line
                WITH line, linenumber() AS index
                CALL (line, index) {
                  MERGE (p:Node {id: index - 2})
                  SET p.label = apoc.convert.fromJsonList(line["label"])
                  SET p.features = apoc.convert.fromJsonList(line["X"])
                } IN TRANSACTIONS OF 10000 ROWS
                """
        session.run(node_query, file=f"file:///{node_file_name}")
        session.run("CREATE INDEX IF NOT EXISTS FOR (n:Node) ON (n.id)")
        edge_query = """
            LOAD CSV FROM $file AS line
            WITH line, linenumber() AS index
            WHERE index <> 1
            CALL (line) {
                MATCH (s:Node {id: toInteger(line[0])}), (t:Node {id: toInteger(line[1])})
                CREATE (s)-[r:connects]->(t)
            } IN TRANSACTIONS OF 10000 ROWS
            """
        session.run(edge_query, file=f"file:///{edge_file_name}")
        return time.time() - start
        
def read(driver, hops, X, y, edge_index, random_sample_size= 1000):
    overall_run_time = 0     
    test_time = 0     
    with driver.session(database = "neo4j") as session:
        np.random.seed(42)
        seed_node_ids = np.random.choice(np.arange(X_and_y.shape[0]), size = random_sample_size, replace = False)
        for seed_node_id in tqdm(seed_node_ids):
            start = time.time()
            subgraph_query = f"""
                MATCH (t {{id: $seed_node_id}})
                MATCH (s)-[r*0..{hops}]->(t)
                UNWIND r AS edge
                WITH t, edge
                WITH 
                  collect(DISTINCT [startNode(edge).id, endNode(edge).id]) AS edges,
                  collect(DISTINCT startNode(edge)) AS startNodes,
                  t AS endNode
                UNWIND (startNodes + [endNode]) AS node
                WITH edges, COLLECT(DISTINCT node) AS uniqueNodes
                UNWIND uniqueNodes AS node
                WITH edges, node, node.features AS nodeFeatures
                ORDER BY node.id
                RETURN 
                  edges,
                  collect(node.id) AS idCollection,
                  collect(node.label) AS labels,
                  collect(nodeFeatures) AS features
                """
            results = session.run(subgraph_query, seed_node_id = seed_node_id)
            res_df = results.to_df()
            if res_df.empty: continue
            subgraph_edge_index = np.array(res_df["edges"][0]).transpose()        
            node_ids = np.array(res_df["idCollection"][0])
            _, cols_source = np.nonzero((subgraph_edge_index[0] == node_ids[:, None]).transpose())
            _, cols_target = np.nonzero((subgraph_edge_index[1] == node_ids[:, None]).transpose())
            
            remapped_edge_index = np.concatenate([np.expand_dims(cols_source, axis = 0), np.expand_dims(cols_target, axis = 0)], axis = 0)
            features =np.array(res_df["features"][0])
            labels = np.array(res_df["labels"][0])       
            overall_run_time += time.time() - start    
            
            ## Testing
            start = time.time()
            remapped_edge_index_test, features_test, labels_test, _ = get_subgraph_from_in_mem_graph_optimized(X, y, seed_node_id, edge_index, hops)        
            test_time += time.time() - start
            assert (sort_edge_index(torch.from_numpy(remapped_edge_index_test)) == sort_edge_index(torch.from_numpy(remapped_edge_index))).sum() / (remapped_edge_index_test.shape[-1] * remapped_edge_index_test.shape[0]), "Edges doesnt match"
            assert np.allclose(labels_test, labels), "Labels does not match"
            assert np.allclose(features, features_test), "features doe not match"
        return overall_run_time, test_time

def update_nodes(driver, X_and_y, X, y, random_sample_size = 1000):
    with driver.session(database = "neo4j") as session:
        np.random.seed(42)
        node_ids = np.random.choice(np.arange(X_and_y.shape[0]), size = random_sample_size, replace = False).tolist()
        start = time.time()
        for node_id in tqdm(node_ids):
            np.random.seed(42)
            features = np.random.rand(X.shape[-1]).tolist()  # Random values between 0 and 1
            labels = np.random.randint(0, 2, size=y.shape[-1]).tolist()  # Adjust label range as needed            
            cypher_query = """
            MATCH (n:Node {id: $node_id})
            SET n.features = $features, n.label = $labels
            """
            session.run(cypher_query, node_id=node_id, features=features, labels=labels)
        return time.time() - start

def update_edges(driver, edge_index, X_and_y, random_sample_size = 1000):
    np.random.seed(42)
    edge_ids = np.random.choice(np.arange(edge_index.shape[-1]),
                                size=random_sample_size,
                                replace=False).tolist()
    selected_edges = edge_index[:, edge_ids].transpose(-1, 0)
    with driver.session() as session:
        start = time.time()
        for selected_edge in tqdm(selected_edges, desc="Updating edges"):
            source_id, target_id = selected_edge
            np.random.seed(42)
            new_target_id = int(np.random.randint(0, X_and_y.shape[0]))
            
            # Cypher query to update the target_id of the relationship that matches both source_id and target_id.
            cypher_query = """
            MATCH (s {id: $source_id}), (t {id: $target_id}), (new_t {id: $new_target_id})
            MATCH (s)-[r:connects]->(t)
            DELETE r
            CREATE (s)-[new_r:connects]->(new_t)
            """
            session.run(cypher_query,
                        source_id=int(source_id),
                        target_id=int(target_id),
                        new_target_id=new_target_id)
    
    return time.time() - start
    

def delete(driver):
    start = time.time()
    driver.execute_query("MATCH (n) DETACH DELETE(n)")
    return time.time() - start

In [29]:
output_df = pd.DataFrame(columns = ["name", "create", "read", "read_mem", "update_nodes", "update_edges", "delete"])

for num_nodes in tqdm([1_000, 10_000, 100_000]):
    for num_edges in tqdm(["5_edges", "10_edges", "20_edges", "scale_free"]):
        feature_file_name = f"X_{str(num_nodes)}_nodes_{num_edges}.csv"
        label_file_name = f"y_{str(num_nodes)}_nodes_{num_edges}.csv"
        edge_file_name = f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv"
        assert os.path.exists(f"syn_data/{feature_file_name}"), "Feature file does not exist"
        assert os.path.exists(f"syn_data/{label_file_name}"), "Label file does not exist"
        assert os.path.exists(f"syn_data/{edge_file_name}"), "Edge file does not exist"
        
        X = pd.read_csv(f"syn_data/{feature_file_name}")
        y = pd.read_csv(f"syn_data/{label_file_name}")
        edges = pd.read_csv(f"syn_data/{edge_file_name}")
        edges.columns = ["source_id", "target_id"]
        edge_index = edges.values.transpose(-1, 0)
        node_file_name = f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_neo4j.csv"
        if not os.path.exists(f"syn_data/{node_file_name}"):
            X_and_y = pd.DataFrame()
            X_and_y["X"] = X.apply(lambda row: [row[column] for column in X.columns], axis=1)
            X_and_y["label"] = y.apply(lambda row: [int(row[column]) for column in y.columns], axis=1)
            X_and_y.to_csv(f"syn_data/{node_file_name}", sep = ",", index = True)
        else:
            X_and_y = pd.read_csv(f"syn_data/{node_file_name}", sep = ",", index_col = 0)
        create_time = create(driver, node_file_name, edge_file_name)
        read_times = dict()
        read_times_mem = dict()
        for hops in tqdm(range(1, 4)):
            read_time, read_time_mem = read(driver, hops, X, y, edge_index, random_sample_size= 1000)
            read_times[hops] = read_time
            read_times_mem[hops] = read_time_mem
        update_time_nodes = update_nodes(driver, X_and_y, X, y, random_sample_size = 1000)
        update_time_edges = update_edges(driver, edge_index, X_and_y, random_sample_size = 1000)
        delete_time = delete(driver)
        new_row_dict = {"name": f"{str(num_nodes)}_nodes_{num_edges}", "create": create_time, "update_nodes": update_time_nodes, "update_edges": update_time_edges, "delete": delete_time}
        for hops in read_times:
            new_row_dict[f"read_{hops}"] = read_times[hops]
            new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
        new_row = pd.DataFrame([new_row_dict])
        output_df = pd.concat((output_df, new_row), ignore_index=True)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Updating edges:   0%|          | 0/1000 [00:00<?, ?it/s]

In [31]:
output_df.to_csv("neo4j_first_res.csv")

In [32]:
output_df

,name,create,read,read_mem,update_nodes,update_edges,delete
0,1000_nodes_5_edges,0.334576,44.488036,1.197679,10.297816,18.144025,0.100055
1,1000_nodes_10_edges,0.544140,125.673550,1.183891,10.444046,18.287151,0.197124
2,1000_nodes_20_edges,0.516626,266.784588,2.175910,9.988736,17.376780,0.248055
3,1000_nodes_scale_free,0.171835,12.821134,0.403466,9.225158,16.314971,0.062260
4,10000_nodes_5_edges,2.176680,48.771195,2.879810,9.512965,21.926965,0.730787
5,10000_nodes_10_edges,2.998675,188.326851,3.944483,10.377921,19.426703,0.729021
6,10000_nodes_20_edges,4.020414,1141.811181,7.923205,9.078572,18.742524,2.118316
7,10000_nodes_scale_free,1.106992,65.087809,0.769648,8.351108,18.112133,0.256757
8,100000_nodes_5_edges,16.246277,70.256942,18.781849,9.334024,51.801995,5.609946
9,100000_nodes_10_edges,27.178283,215.028669,25.793967,10.242667,53.863213,8.649685
